<a href="https://colab.research.google.com/github/choki0715/lecture/blob/master/quant_perform.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# answer_quality_comparison.py

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch
import numpy as np
from datasets import load_dataset
import json
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import pandas as pd
from typing import List, Dict
import warnings
warnings.filterwarnings('ignore')

class QualityEvaluator:
    """양자화 모델 답변 품질 평가"""

    def __init__(self, model_name="gpt2"):
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        self.results = {}

    def load_models(self):
        """다양한 양자화 모델 로드"""
        models = {}

        # FP16 (기준 모델)
        print("Loading FP16 model...")
        models['FP16'] = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            torch_dtype=torch.float16,
            device_map="auto"
        )

        # INT8
        print("Loading INT8 model...")
        models['INT8'] = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            load_in_8bit=True,
            device_map="auto"
        )

        # INT4
        print("Loading INT4 model...")
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_compute_dtype=torch.float16
        )
        models['INT4'] = AutoModelForCausalLM.from_pretrained(
            self.model_name,
            quantization_config=bnb_config,
            device_map="auto"
        )

        return models

    def generate_text(self, model, prompt, max_length=100, num_samples=1, temperature=0.7):
        """텍스트 생성"""
        inputs = self.tokenizer(prompt, return_tensors="pt", padding=True)
        inputs = {k: v.to(model.device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=max_length,
                temperature=temperature,
                do_sample=True if temperature > 0 else False,
                num_return_sequences=num_samples,
                pad_token_id=self.tokenizer.eos_token_id
            )

        generated_texts = []
        for output in outputs:
            text = self.tokenizer.decode(output[inputs['input_ids'].shape[1]:], skip_special_tokens=True)
            generated_texts.append(text)

        return generated_texts[0] if num_samples == 1 else generated_texts

    def test_factual_accuracy(self, models):
        """사실 정확도 테스트"""
        print("\n📊 사실 정확도 테스트")
        print("="*50)

        factual_prompts = [
            ("The capital of France is", ["Paris"]),
            ("Water boils at", ["100", "degrees", "Celsius", "212", "Fahrenheit"]),
            ("The largest planet in our solar system is", ["Jupiter"]),
            ("The speed of light is approximately", ["299", "300", "thousand", "kilometers"]),
            ("The author of Romeo and Juliet is", ["Shakespeare", "William"]),
            ("The chemical symbol for gold is", ["Au"]),
            ("The square root of 144 is", ["12", "twelve"]),
            ("The currency of Japan is", ["yen", "Yen", "JPY"]),
            ("The smallest prime number is", ["2", "two"]),
            ("The freezing point of water is", ["0", "zero", "32", "Celsius", "Fahrenheit"])
        ]

        results = {}

        for model_name, model in models.items():
            correct = 0
            total = len(factual_prompts)

            print(f"\n{model_name}:")
            for prompt, expected_words in factual_prompts:
                generated = self.generate_text(model, prompt, max_length=20, temperature=0.1)

                # 정답 체크
                is_correct = any(word.lower() in generated.lower() for word in expected_words)
                if is_correct:
                    correct += 1
                    print(f"  ✅ {prompt} → {generated[:50]}")
                else:
                    print(f"  ❌ {prompt} → {generated[:50]}")

            accuracy = correct / total
            results[model_name] = accuracy
            print(f"  정확도: {accuracy:.1%}")

        return results

    def test_consistency(self, models):
        """일관성 테스트 (같은 프롬프트, 여러 번 생성)"""
        print("\n📊 일관성 테스트")
        print("="*50)

        test_prompt = "The most important quality of a leader is"
        num_generations = 5

        consistency_scores = {}

        for model_name, model in models.items():
            print(f"\n{model_name}:")

            generations = []
            for i in range(num_generations):
                text = self.generate_text(model, test_prompt, max_length=50, temperature=0.3)
                generations.append(text)
                print(f"  {i+1}: {text[:80]}")

            # 일관성 점수 계산 (ROUGE-L 사용)
            scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
            scores = []

            for i in range(len(generations)):
                for j in range(i+1, len(generations)):
                    score = scorer.score(generations[i], generations[j])
                    scores.append(score['rougeL'].fmeasure)

            consistency_score = np.mean(scores) if scores else 0
            consistency_scores[model_name] = consistency_score
            print(f"  일관성 점수: {consistency_score:.3f}")

        return consistency_scores

    def test_coherence(self, models):
        """논리적 일관성 테스트"""
        print("\n📊 논리적 일관성 테스트")
        print("="*50)

        coherence_prompts = [
            "If it's raining outside, you should take an umbrella because",
            "To bake a cake, first you need to",
            "The student failed the exam, therefore",
            "Exercise is important for health because",
            "Before going to sleep, it's good to"
        ]

        coherence_scores = {}

        for model_name, model in models.items():
            print(f"\n{model_name}:")
            logical_scores = []

            for prompt in coherence_prompts:
                generated = self.generate_text(model, prompt, max_length=50, temperature=0.5)
                print(f"  {prompt[:30]}... → {generated[:60]}")

                # 간단한 논리성 체크 (키워드 기반)
                logical_keywords = {
                    "raining": ["wet", "dry", "protect", "water", "rain"],
                    "bake": ["ingredients", "mix", "oven", "recipe", "prepare"],
                    "failed": ["study", "try", "next", "improve", "learn"],
                    "Exercise": ["health", "body", "fitness", "energy", "strong"],
                    "sleep": ["relax", "rest", "calm", "routine", "peaceful"]
                }

                for key, keywords in logical_keywords.items():
                    if key in prompt:
                        score = 1.0 if any(kw in generated.lower() for kw in keywords) else 0.0
                        logical_scores.append(score)
                        break

            coherence_scores[model_name] = np.mean(logical_scores)
            print(f"  논리성 점수: {coherence_scores[model_name]:.2f}")

        return coherence_scores

    def test_creativity(self, models):
        """창의성 테스트 (다양성 측정)"""
        print("\n📊 창의성/다양성 테스트")
        print("="*50)

        creative_prompt = "In a world where gravity works backwards,"
        num_samples = 5

        diversity_scores = {}

        for model_name, model in models.items():
            print(f"\n{model_name}:")

            generations = []
            unique_words = set()

            for i in range(num_samples):
                text = self.generate_text(model, creative_prompt, max_length=60, temperature=0.9)
                generations.append(text)
                unique_words.update(text.lower().split())
                print(f"  {i+1}: {text[:80]}")

            # 다양성 점수 (unique words / total words)
            total_words = sum(len(g.split()) for g in generations)
            diversity_score = len(unique_words) / total_words if total_words > 0 else 0
            diversity_scores[model_name] = diversity_score
            print(f"  다양성 점수: {diversity_score:.3f}")

        return diversity_scores

    def test_perplexity(self, models):
        """Perplexity 측정 (낮을수록 좋음)"""
        print("\n📊 Perplexity 측정")
        print("="*50)

        # WikiText-2 데이터셋 샘플 사용
        test_texts = [
            "The history of artificial intelligence began in antiquity, with myths, stories and rumors of artificial beings endowed with intelligence.",
            "Machine learning is a method of data analysis that automates analytical model building.",
            "Natural language processing is a subfield of linguistics, computer science, and artificial intelligence.",
            "Deep learning is part of a broader family of machine learning methods based on artificial neural networks.",
            "Computer vision is an interdisciplinary scientific field that deals with how computers gain understanding from digital images."
        ]

        perplexity_scores = {}

        for model_name, model in models.items():
            total_loss = 0
            total_tokens = 0

            for text in test_texts:
                inputs = self.tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
                inputs = {k: v.to(model.device) for k, v in inputs.items()}

                with torch.no_grad():
                    outputs = model(**inputs, labels=inputs['input_ids'])
                    total_loss += outputs.loss.item() * inputs['input_ids'].shape[1]
                    total_tokens += inputs['input_ids'].shape[1]

            perplexity = np.exp(total_loss / total_tokens)
            perplexity_scores[model_name] = perplexity
            print(f"{model_name}: {perplexity:.2f}")

        return perplexity_scores

    def test_qa_capability(self, models):
        """질문-답변 능력 테스트"""
        print("\n📊 Q&A 능력 테스트")
        print("="*50)

        qa_pairs = [
            ("Q: What is 15 + 27?\nA:", ["42"]),
            ("Q: What color is the sky?\nA:", ["blue", "Blue"]),
            ("Q: How many days are in a week?\nA:", ["7", "seven", "Seven"]),
            ("Q: What is the opposite of hot?\nA:", ["cold", "Cold"]),
            ("Q: What animal is known as man's best friend?\nA:", ["dog", "Dog"])
        ]

        qa_scores = {}

        for model_name, model in models.items():
            correct = 0
            print(f"\n{model_name}:")

            for question, expected in qa_pairs:
                answer = self.generate_text(model, question, max_length=20, temperature=0.1)
                is_correct = any(exp in answer for exp in expected)

                if is_correct:
                    correct += 1
                    print(f"  ✅ {question.split('?')[0]}? → {answer[:30]}")
                else:
                    print(f"  ❌ {question.split('?')[0]}? → {answer[:30]}")

            qa_scores[model_name] = correct / len(qa_pairs)
            print(f"  정답률: {qa_scores[model_name]:.1%}")

        return qa_scores

    def run_all_tests(self):
        """모든 테스트 실행"""
        print("🚀 양자화 모델 답변 품질 종합 평가")
        print("="*60)

        # 모델 로드
        models = self.load_models()

        # 각 테스트 실행
        results = {
            'factual_accuracy': self.test_factual_accuracy(models),
            'consistency': self.test_consistency(models),
            'coherence': self.test_coherence(models),
            'creativity': self.test_creativity(models),
            'perplexity': self.test_perplexity(models),
            'qa_capability': self.test_qa_capability(models)
        }

        # 결과 요약
        self.print_summary(results)

        # CSV로 저장
        self.save_results(results)

        return results

    def print_summary(self, results):
        """결과 요약 출력"""
        print("\n" + "="*60)
        print("📊 종합 평가 결과")
        print("="*60)

        # DataFrame 생성
        df = pd.DataFrame(results)

        # 점수 정규화 (Perplexity는 낮을수록 좋으므로 역수)
        df_normalized = df.copy()
        df_normalized['perplexity'] = 1 / df_normalized['perplexity']

        # 종합 점수 계산
        df_normalized['total_score'] = df_normalized.mean(axis=1)

        print("\n📈 각 항목별 점수:")
        print(df.round(3).to_string())

        print("\n🏆 종합 순위:")
        ranking = df_normalized['total_score'].sort_values(ascending=False)
        for i, (model, score) in enumerate(ranking.items(), 1):
            print(f"{i}. {model}: {score:.3f}")

        # FP16 대비 품질 저하율
        print("\n📉 FP16 대비 품질 변화:")
        baseline = df.loc['FP16']
        for model in df.index:
            if model != 'FP16':
                diff = df.loc[model] - baseline
                avg_diff = diff.mean()
                print(f"{model}: {avg_diff:+.3f} (평균)")

    def save_results(self, results):
        """결과 저장"""
        df = pd.DataFrame(results)
        df.to_csv('quality_evaluation_results.csv')

        # JSON으로도 저장
        with open('quality_evaluation_results.json', 'w') as f:
            json.dump(results, f, indent=2)

        print("\n💾 결과가 저장되었습니다:")
        print("  - quality_evaluation_results.csv")
        print("  - quality_evaluation_results.json")

# 실행
if __name__ == "__main__":
    evaluator = QualityEvaluator(model_name="gpt2")
    results = evaluator.run_all_tests()